## Imports

In [1]:
import pandas as pd
import numpy as np
import os
import folium
import ast
import textwrap
import googlemaps
from folium.plugins import Fullscreen, MiniMap

## Data read-in

In [13]:
df = pd.read_csv("sale_data.csv",index_col=0,dtype='str')

df['Recent notable sales / listings'] = df['Recent notable sales / listings'].str.replace(r'\$', r'\$', regex=True)

df

,Address,Status,Recent notable sales / listings,Sponsor ppsqf,Resale ppsqf,Price change (%)
0,250 West Street,Resale,\$60 million four-unit spread,NaN,NaN,NaN
1,150 Charles,Resale,"Unit 9A sold for \$60 million (\$10,300 ppsqf)","$3,576","$5,213",45.78%
2,70 Vestry Street,Resale,NaN,"$3,808","$4,655",22.24%
3,443 Greenwich Street,Resale,NaN,"$3,389","$4,505",32.93%
4,140 Jane Street,Sponsor - Boutique,"\$87.5 million PH listed (\$9,180 ppsqf)","$5,470",NaN,NaN
5,125 Perry Street,Sponsor - Boutique,"\$57.5 million PH in contract (\$8,088 ppsqf)","$6,066",NaN,NaN
6,500 West 18th Street,Sponsor - tower,"\$47 million PH sold (\$6,780 ppsqf)","$3,348",NaN,NaN
7,80 Clarkson Street,Sponsor - tower,"\$63 million listing (\$9,083 ppsqf)","$5,809",NaN,NaN
8,551 West 21st Street,Resale,"\$75 million listing (\$11,718 ppsqf)",NaN,NaN,NaN


In [14]:
df['full_address'] = df['Address'] + ' New York, NY'

In [16]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

%store -r map_box_api_key

In [17]:
# Define the geocode function
def geocode(add):
    g = gmaps_key.geocode(add)
    if g:
        lat = g[0]["geometry"]["location"]["lat"]
        lng = g[0]["geometry"]["location"]["lng"]
        return (lat, lng)
    else:
        return None

# Apply geocoding to the 'geo_address' column and store the results in 'geocoded' column
df['geocoded'] = df['full_address'].apply(geocode)

In [18]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].apply(lambda x: (None, None) if x == 'None' else x.strip('()').split(', ', 1)).apply(pd.Series)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

In [25]:
df.columns

Index(['Address', 'Status', 'Recent notable sales / listings', 'Sponsor ppsqf',
       'Resale ppsqf', 'Price change (%)', 'full_address', 'geocoded', 'lat',
       'lon'],
      dtype='object')

In [21]:
map_df = df

In [26]:
import folium
from folium.plugins import Fullscreen, MiniMap
from collections import defaultdict
from branca.element import MacroElement
from jinja2 import Template

# ─── 1. Your DataFrame ────────────────────────────────────────────────────────
# map_df already has these cols: Address, Status, Recent notable sales / listings,
# full_address, lat, lon, etc.

# ─── 2. Build a color map by Status ───────────────────────────────────────────
_palette = [
    'red','blue','green','purple','orange','darkred','lightred','beige',
    'darkblue','darkgreen','cadetblue','darkpurple','pink','lightblue',
    'lightgreen','gray','black','lightgray'
]
unique_statuses = map_df['Status'].unique()
status_color = {
    st: _palette[i % len(_palette)]
    for i, st in enumerate(unique_statuses)
}

# ─── 3. Popup HTML ────────────────────────────────────────────────────────────
def create_popup(row):
    # list of (column_name, label) pairs in the order you want them
    fields = [
        ('Address', 'Address'),
        ('Status', 'Status'),
        ('Recent notable sales / listings', 'Notable'),
        ('Sponsor ppsqf', 'Sponsor ppsqf'),
        ('Resale ppsqf', 'Resale ppsqf'),
        ('Price change (%)', 'Price change'),
    ]

    html = ['<div style="min-width:220px;font-size:13px;">']
    for col, label in fields:
        val = row.get(col)
        if pd.notnull(val):
            html.append(f'<strong>{label}:</strong> {val}<br>')
    html.append('</div>')
    
    return ''.join(html)

# ─── 4. Initialize the map ────────────────────────────────────────────────────
center = [map_df['lat'].mean(), map_df['lon'].mean()]
m = folium.Map(
    location=center,
    zoom_start=13,
    tiles=None,            # or use 'cartodbpositron'
    scrollWheelZoom=False
)

# ─── 5. Add Mapbox (or fallback) ─────────────────────────────────────────────
folium.TileLayer(
    tiles=(
        "https://api.mapbox.com/styles/v1/"
        "mapbox/streets-v11/tiles/256/{z}/{x}/{y}@2x"
        f"?access_token={map_box_api_key}"
    ),
    attr="Mapbox",
    name="Streets (Mapbox)",
    overlay=False,
    control=False,
    show=True
).add_to(m)

# ─── 6. Plot your points ─────────────────────────────────────────────────────
for _, row in map_df.iterrows():
    col = status_color[row['Status']]
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=6,
        color=col,
        fill=True, fill_color=col, fill_opacity=0.7,
        popup=folium.Popup(create_popup(row), max_width=250)
    ).add_to(m)

# ─── 7. Legend ───────────────────────────────────────────────────────────────
legend_html = """
{% macro html(this, kwargs) %}
  <div style="
     position: fixed;
     bottom: 30px; left: 30px;
     width: 140px; background: white;
     border:1px solid #888; z-index:9999;
     font-size:13px; padding: 8px;">
    <b>Status legend</b><br>
    {% for st, col in this.mapping.items() %}
      <i style="background:{{col}};width:12px;height:12px;
                display:inline-block;margin-right:6px;"></i>
      {{st}}<br>
    {% endfor %}
  </div>
{% endmacro %}
"""
legend = MacroElement()
legend._template = Template(legend_html)
legend.mapping = status_color
m.get_root().add_child(legend)

# ─── 8. Controls & display ─────────────────────────────────────────────────
Fullscreen().add_to(m)
MiniMap(toggle_display=True).add_to(m)

# In a notebook you’ll see the map; in a script you can do:
# m.save("status_map.html")
m

## Grab URL